In [1]:
pip install patool

     -------------------------------------- 77.5/77.5 kB 116.4 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install pyunpack

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install openpyxl


     ------------------------------------- 250.0/250.0 kB 48.4 kB/s eta 0:00:00


In [5]:
from pyunpack import Archive
from pathlib import Path

Archive("Data_database1.rar").extractall('Data_database')

In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)

In [2]:
combined_df = pd.DataFrame()

In [3]:
import pandas as pd
import os

file_paths = []

for root, dirs, files in os.walk('Data_database'):
    for file in files:
        if file.endswith('.xlsx'):
           file_paths.append(os.path.join(root, file))

In [4]:
print(len(file_paths))

3275


In [5]:
combined_data = []
error_files = []
for file_path in file_paths:
    try:
        df = pd.read_excel(file_path, engine='openpyxl')
    except:
        error_files.append(file_path)
    # transposed_df = df.T  # Transpose the DataFrame
    # combined_data.append(transposed_df)
print(error_files)

['Data_database\\Data_database1\\types\\~$algae-chlorella_#1384.xlsx', 'Data_database\\Data_database1\\types\\~$algae-Synechococcus_#2327.xlsx', 'Data_database\\Data_database1\\types\\grass\\plant\\hemp\\~$hemp-(Cannabis-sativa-L.)-whole-plant_#1702.xlsx']


In [6]:
print(len(error_files), error_files)

3 ['Data_database\\Data_database1\\types\\~$algae-chlorella_#1384.xlsx', 'Data_database\\Data_database1\\types\\~$algae-Synechococcus_#2327.xlsx', 'Data_database\\Data_database1\\types\\grass\\plant\\hemp\\~$hemp-(Cannabis-sativa-L.)-whole-plant_#1702.xlsx']


In [7]:
print("Data_database\\Data_database1\\types\\barley_#946.xlsx" in file_paths)
print('Data_database\\Data_database1\\types\\grass\\plant\\hemp\\hemp-(Cannabis-sativa-L.)-whole-plant_#1702.xlsx' in file_paths)

True
True


In [116]:
df1 = pd.read_excel(file_paths[0], engine='openpyxl')

In [128]:
updf = df1.iloc[:df1.index[df1.iloc[:, 0] == "Values"][0], :]
downdf = df1.iloc[df1.index[df1.iloc[:, 0] == "Values"][0]:, :]

In [129]:
updf_T = updf.T.iloc[0:2]

In [130]:
# drop columns that have NaN in the two rows of up, renumber the columns
updf_T = updf_T.dropna(axis=1, how='all')
updf_T.columns = range(updf_T.shape[1])
updf_T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
Phyllis2,https://phyllis.nl/Biomass/View/3847,Acinetospora crinita (#3847),ID-number,Material,Description,Classification,Submitter organisation,Submission date,Remarks,NaN,NaN,NaN,NaN,NaN,NaN,Literature,NaN,NaN
Unnamed: 1,NaN,NaN,#3847,Acinetospora crinita,Freeze dried,ECN Phyllis classification > Marine biomass (a...,TNO (Netherlands),2023-02-15 08:37:39,Phylum: Ochrophyta,Genus: Acinetospora,Species: Acinetospora crinita,Generic algae family: Brown,Cultivation region: Istra Peninsula (Croatia) ...,Wild/culture: Wild,Harvesting date: Jun-04,Year: 2016,Journal/project: Phycological Society of America,DOI/link: DOI: 10.1111/jpy.12413


In [131]:
updf_T = updf_T.drop([0,1], axis=1)
updf_T.columns = range(updf_T.shape[1])


In [132]:
# forward fill the NaN values in the first row
updf_T = updf_T.fillna(method='ffill', axis=1)
updf_T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
Phyllis2,ID-number,Material,Description,Classification,Submitter organisation,Submission date,Remarks,Remarks,Remarks,Remarks,Remarks,Remarks,Remarks,Literature,Literature,Literature
Unnamed: 1,#3847,Acinetospora crinita,Freeze dried,ECN Phyllis classification > Marine biomass (a...,TNO (Netherlands),2023-02-15 08:37:39,Phylum: Ochrophyta,Genus: Acinetospora,Species: Acinetospora crinita,Generic algae family: Brown,Cultivation region: Istra Peninsula (Croatia) ...,Wild/culture: Wild,Harvesting date: Jun-04,Year: 2016,Journal/project: Phycological Society of America,DOI/link: DOI: 10.1111/jpy.12413


In [134]:
#create a mapping for the first row and the second row of updf_T, if the keys are the same merge the values
updf_T_dict = dict()
# iterate through the first row of updf_T let them be the keys of the dictionary if it is not already in the dictionary, if it is already in the dictionary, merge the values
for i, col in enumerate(updf_T.iloc[0]):
    if col not in updf_T_dict:
        updf_T_dict[col] = updf_T.iloc[1, i]
    else:
        updf_T_dict[col] = updf_T_dict[col] + ' ^ ' + updf_T.iloc[1, i]

In [135]:
updf_T_dict

{'ID-number': '#3847',
 'Material': 'Acinetospora crinita',
 'Description': 'Freeze dried',
 'Classification': 'ECN Phyllis classification > Marine biomass (algae) > Macroalgae > Acinetospora crinita',
 'Submitter organisation': 'TNO (Netherlands)',
 'Submission date': datetime.datetime(2023, 2, 15, 8, 37, 39),
 'Remarks': 'Phylum: Ochrophyta ^ Genus: Acinetospora ^ Species: Acinetospora crinita ^ Generic algae family: Brown ^ Cultivation region: Istra Peninsula (Croatia) + Tremiti Islands (Italy) ^ Wild/culture: Wild ^ Harvesting date: Jun-04',
 'Literature': 'Year: 2016 ^ Journal/project: Phycological Society of America ^ DOI/link: DOI: 10.1111/jpy.12413'}

In [141]:
# create a new dataframe with the dictionary with keys as columns and values as rows
updf_T_new = pd.DataFrame(updf_T_dict, index=[0])
updf_T_new


,ID-number,Material,Description,Classification,Submitter organisation,Submission date,Remarks,Literature
0,#3847,Acinetospora crinita,Freeze dried,ECN Phyllis classification > Marine biomass (a...,TNO (Netherlands),2023-02-15 08:37:39,Phylum: Ochrophyta ^ Genus: Acinetospora ^ Spe...,Year: 2016 ^ Journal/project: Phycological Soc...


In [69]:
downdf_T = downdf.T
downdf_T.columns = range(downdf_T.shape[1])
downdf_T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
Phyllis2,Values,Property,NaN,Main biomass properties,Biochemical analysis,Fatty acids,C14:0 Myristic,C15:0 Pentadecylic,C16:0 Palmitic,C16:1ω7 Palmitoleic,C18:0 Stearic,C18:1ω9 Oleic,C18:2ω6 Linoleic,C18:3ω3 α-Linolenic,C18:3ω6 γ-Linolenic,C20:0 Arachidic,C20:4ω6 Arachidonic,"C20:5ω3 (Eicosapentaenoic, EPA)",C22:1,C22:6ω3 (DHA)
Unnamed: 1,NaN,Unit,NaN,NaN,NaN,NaN,wt% (dry),wt% (dry),wt% (dry),wt% (dry),wt% (dry),wt% (dry),wt% (dry),wt% (dry),wt% (dry),wt% (dry),wt% (dry),wt% (dry),wt% (dry),wt% (dry)
Unnamed: 2,NaN,Value,ar,NaN,NaN,NaN,14.82,1.31,34.15,16.19,2.76,7.14,4.36,5.06,0.58,0.57,2.26,3.49,0.95,0.04
Unnamed: 3,NaN,NaN,dry,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Unnamed: 4,NaN,NaN,daf,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Unnamed: 5,NaN,Std dev,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Unnamed: 6,NaN,Det lim,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Unnamed: 7,NaN,Lab,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Unnamed: 8,NaN,Date,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Unnamed: 9,NaN,Method,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [70]:
# forward fill the NaN values in the first column
downdf_T[1] = downdf_T[1].fillna(method='ffill', axis=0)

In [71]:
# concatenate two columns [1, 2] into one if one of the row is nan ignore the nan, drop the two columns, rename the new column as 0, renunmber the columns
downdf_T[1] = downdf_T[1].fillna('') + '_' + downdf_T[2].fillna('')
downdf_T = downdf_T.drop([2], axis=1)
downdf_T.columns = range(downdf_T.shape[1])
downdf_T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
Phyllis2,Values,Property_,Main biomass properties,Biochemical analysis,Fatty acids,C14:0 Myristic,C15:0 Pentadecylic,C16:0 Palmitic,C16:1ω7 Palmitoleic,C18:0 Stearic,C18:1ω9 Oleic,C18:2ω6 Linoleic,C18:3ω3 α-Linolenic,C18:3ω6 γ-Linolenic,C20:0 Arachidic,C20:4ω6 Arachidonic,"C20:5ω3 (Eicosapentaenoic, EPA)",C22:1,C22:6ω3 (DHA)
Unnamed: 1,NaN,Unit_,NaN,NaN,NaN,wt% (dry),wt% (dry),wt% (dry),wt% (dry),wt% (dry),wt% (dry),wt% (dry),wt% (dry),wt% (dry),wt% (dry),wt% (dry),wt% (dry),wt% (dry),wt% (dry)
Unnamed: 2,NaN,Value_ar,NaN,NaN,NaN,14.82,1.31,34.15,16.19,2.76,7.14,4.36,5.06,0.58,0.57,2.26,3.49,0.95,0.04
Unnamed: 3,NaN,Value_dry,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Unnamed: 4,NaN,Value_daf,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Unnamed: 5,NaN,Std dev_,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Unnamed: 6,NaN,Det lim_,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Unnamed: 7,NaN,Lab_,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Unnamed: 8,NaN,Date_,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Unnamed: 9,NaN,Method_,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [88]:
downdf_ = downdf_T.copy()

In [89]:
# make all entries in the first row to be the column names
downdf_.columns = downdf_.iloc[0]

# drop the first and second row using the index
downdf_ = downdf_.drop(downdf_.index[0:2])


# drop the columns that have all NaN values
downdf_ = downdf_.dropna(axis=1, how='all')
downdf_

Phyllis2,Property_,C14:0 Myristic,C15:0 Pentadecylic,C16:0 Palmitic,C16:1ω7 Palmitoleic,C18:0 Stearic,C18:1ω9 Oleic,C18:2ω6 Linoleic,C18:3ω3 α-Linolenic,C18:3ω6 γ-Linolenic,C20:0 Arachidic,C20:4ω6 Arachidonic,"C20:5ω3 (Eicosapentaenoic, EPA)",C22:1,C22:6ω3 (DHA)
Unnamed: 2,Value_ar,14.82,1.31,34.15,16.19,2.76,7.14,4.36,5.06,0.58,0.57,2.26,3.49,0.95,0.04
Unnamed: 3,Value_dry,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Unnamed: 4,Value_daf,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Unnamed: 5,Std dev_,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Unnamed: 6,Det lim_,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Unnamed: 7,Lab_,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Unnamed: 8,Date_,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Unnamed: 9,Method_,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Unnamed: 10,Remarks_,Percentage of total fatty acid content,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [94]:
prefix_col = downdf_['Property_'].values.tolist()

In [95]:
parameter_col = downdf_.columns[1:]

In [109]:
# create a dataframe from downdf_ by indexing the columns with parameter_col and the rows with prefix_col with the columns names as "prefix_col+ '_' + parameter_col"
dict_ = dict()
for row in prefix_col:
    for col in parameter_col:
        dict_[row + '_' + col] = downdf_[downdf_['Property_'] == row][col].values[0]


In [114]:
# create the dataframe from the dictionary
downdf_new = pd.DataFrame(dict_, index=[0])
downdf_new

,Value_ar_C14:0 Myristic,Value_ar_C15:0 Pentadecylic,Value_ar_C16:0 Palmitic,Value_ar_C16:1ω7 Palmitoleic,Value_ar_C18:0 Stearic,Value_ar_C18:1ω9 Oleic,Value_ar_C18:2ω6 Linoleic,Value_ar_C18:3ω3 α-Linolenic,Value_ar_C18:3ω6 γ-Linolenic,Value_ar_C20:0 Arachidic,Value_ar_C20:4ω6 Arachidonic,"Value_ar_C20:5ω3 (Eicosapentaenoic, EPA)",Value_ar_C22:1,Value_ar_C22:6ω3 (DHA),Value_dry_C14:0 Myristic,Value_dry_C15:0 Pentadecylic,Value_dry_C16:0 Palmitic,Value_dry_C16:1ω7 Palmitoleic,Value_dry_C18:0 Stearic,Value_dry_C18:1ω9 Oleic,Value_dry_C18:2ω6 Linoleic,Value_dry_C18:3ω3 α-Linolenic,Value_dry_C18:3ω6 γ-Linolenic,Value_dry_C20:0 Arachidic,Value_dry_C20:4ω6 Arachidonic,"Value_dry_C20:5ω3 (Eicosapentaenoic, EPA)",Value_dry_C22:1,Value_dry_C22:6ω3 (DHA),Value_daf_C14:0 Myristic,Value_daf_C15:0 Pentadecylic,Value_daf_C16:0 Palmitic,Value_daf_C16:1ω7 Palmitoleic,Value_daf_C18:0 Stearic,Value_daf_C18:1ω9 Oleic,Value_daf_C18:2ω6 Linoleic,Value_daf_C18:3ω3 α-Linolenic,Value_daf_C18:3ω6 γ-Linolenic,Value_daf_C20:0 Arachidic,Value_daf_C20:4ω6 Arachidonic,"Value_daf_C20:5ω3 (Eicosapentaenoic, EPA)",Value_daf_C22:1,Value_daf_C22:6ω3 (DHA),Std dev__C14:0 Myristic,Std dev__C15:0 Pentadecylic,Std dev__C16:0 Palmitic,Std dev__C16:1ω7 Palmitoleic,Std dev__C18:0 Stearic,Std dev__C18:1ω9 Oleic,Std dev__C18:2ω6 Linoleic,Std dev__C18:3ω3 α-Linolenic,Std dev__C18:3ω6 γ-Linolenic,Std dev__C20:0 Arachidic,Std dev__C20:4ω6 Arachidonic,"Std dev__C20:5ω3 (Eicosapentaenoic, EPA)",Std dev__C22:1,Std dev__C22:6ω3 (DHA),Det lim__C14:0 Myristic,Det lim__C15:0 Pentadecylic,Det lim__C16:0 Palmitic,Det lim__C16:1ω7 Palmitoleic,Det lim__C18:0 Stearic,Det lim__C18:1ω9 Oleic,Det lim__C18:2ω6 Linoleic,Det lim__C18:3ω3 α-Linolenic,Det lim__C18:3ω6 γ-Linolenic,Det lim__C20:0 Arachidic,Det lim__C20:4ω6 Arachidonic,"Det lim__C20:5ω3 (Eicosapentaenoic, EPA)",Det lim__C22:1,Det lim__C22:6ω3 (DHA),Lab__C14:0 Myristic,Lab__C15:0 Pentadecylic,Lab__C16:0 Palmitic,Lab__C16:1ω7 Palmitoleic,Lab__C18:0 Stearic,Lab__C18:1ω9 Oleic,Lab__C18:2ω6 Linoleic,Lab__C18:3ω3 α-Linolenic,Lab__C18:3ω6 γ-Linolenic,Lab__C20:0 Arachidic,Lab__C20:4ω6 Arachidonic,"Lab__C20:5ω3 (Eicosapentaenoic, EPA)",Lab__C22:1,Lab__C22:6ω3 (DHA),Date__C14:0 Myristic,Date__C15:0 Pentadecylic,Date__C16:0 Palmitic,Date__C16:1ω7 Palmitoleic,Date__C18:0 Stearic,Date__C18:1ω9 Oleic,Date__C18:2ω6 Linoleic,Date__C18:3ω3 α-Linolenic,Date__C18:3ω6 γ-Linolenic,Date__C20:0 Arachidic,Date__C20:4ω6 Arachidonic,"Date__C20:5ω3 (Eicosapentaenoic, EPA)",Date__C22:1,Date__C22:6ω3 (DHA),Method__C14:0 Myristic,Method__C15:0 Pentadecylic,Method__C16:0 Palmitic,Method__C16:1ω7 Palmitoleic,Method__C18:0 Stearic,Method__C18:1ω9 Oleic,Method__C18:2ω6 Linoleic,Method__C18:3ω3 α-Linolenic,Method__C18:3ω6 γ-Linolenic,Method__C20:0 Arachidic,Method__C20:4ω6 Arachidonic,"Method__C20:5ω3 (Eicosapentaenoic, EPA)",Method__C22:1,Method__C22:6ω3 (DHA),Remarks__C14:0 Myristic,Remarks__C15:0 Pentadecylic,Remarks__C16:0 Palmitic,Remarks__C16:1ω7 Palmitoleic,Remarks__C18:0 Stearic,Remarks__C18:1ω9 Oleic,Remarks__C18:2ω6 Linoleic,Remarks__C18:3ω3 α-Linolenic,Remarks__C18:3ω6 γ-Linolenic,Remarks__C20:0 Arachidic,Remarks__C20:4ω6 Arachidonic,"Remarks__C20:5ω3 (Eicosapentaenoic, EPA)",Remarks__C22:1,Remarks__C22:6ω3 (DHA)
0,14.82,1.31,34.15,16.19,2.76,7.14,4.36,5.06,0.58,0.57,2.26,3.49,0.95,0.04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Percentage of total fatty acid content,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [142]:
# combine the two dataframes of updf_T_new and downdf_new
combined_df = pd.concat([updf_T_new, downdf_new], axis=1)
combined_df


,ID-number,Material,Description,Classification,Submitter organisation,Submission date,Remarks,Literature,Value_ar_C14:0 Myristic,Value_ar_C15:0 Pentadecylic,Value_ar_C16:0 Palmitic,Value_ar_C16:1ω7 Palmitoleic,Value_ar_C18:0 Stearic,Value_ar_C18:1ω9 Oleic,Value_ar_C18:2ω6 Linoleic,Value_ar_C18:3ω3 α-Linolenic,Value_ar_C18:3ω6 γ-Linolenic,Value_ar_C20:0 Arachidic,Value_ar_C20:4ω6 Arachidonic,"Value_ar_C20:5ω3 (Eicosapentaenoic, EPA)",Value_ar_C22:1,Value_ar_C22:6ω3 (DHA),Value_dry_C14:0 Myristic,Value_dry_C15:0 Pentadecylic,Value_dry_C16:0 Palmitic,Value_dry_C16:1ω7 Palmitoleic,Value_dry_C18:0 Stearic,Value_dry_C18:1ω9 Oleic,Value_dry_C18:2ω6 Linoleic,Value_dry_C18:3ω3 α-Linolenic,Value_dry_C18:3ω6 γ-Linolenic,Value_dry_C20:0 Arachidic,Value_dry_C20:4ω6 Arachidonic,"Value_dry_C20:5ω3 (Eicosapentaenoic, EPA)",Value_dry_C22:1,Value_dry_C22:6ω3 (DHA),Value_daf_C14:0 Myristic,Value_daf_C15:0 Pentadecylic,Value_daf_C16:0 Palmitic,Value_daf_C16:1ω7 Palmitoleic,Value_daf_C18:0 Stearic,Value_daf_C18:1ω9 Oleic,Value_daf_C18:2ω6 Linoleic,Value_daf_C18:3ω3 α-Linolenic,Value_daf_C18:3ω6 γ-Linolenic,Value_daf_C20:0 Arachidic,Value_daf_C20:4ω6 Arachidonic,"Value_daf_C20:5ω3 (Eicosapentaenoic, EPA)",Value_daf_C22:1,Value_daf_C22:6ω3 (DHA),Std dev__C14:0 Myristic,Std dev__C15:0 Pentadecylic,Std dev__C16:0 Palmitic,Std dev__C16:1ω7 Palmitoleic,Std dev__C18:0 Stearic,Std dev__C18:1ω9 Oleic,Std dev__C18:2ω6 Linoleic,Std dev__C18:3ω3 α-Linolenic,Std dev__C18:3ω6 γ-Linolenic,Std dev__C20:0 Arachidic,Std dev__C20:4ω6 Arachidonic,"Std dev__C20:5ω3 (Eicosapentaenoic, EPA)",Std dev__C22:1,Std dev__C22:6ω3 (DHA),Det lim__C14:0 Myristic,Det lim__C15:0 Pentadecylic,Det lim__C16:0 Palmitic,Det lim__C16:1ω7 Palmitoleic,Det lim__C18:0 Stearic,Det lim__C18:1ω9 Oleic,Det lim__C18:2ω6 Linoleic,Det lim__C18:3ω3 α-Linolenic,Det lim__C18:3ω6 γ-Linolenic,Det lim__C20:0 Arachidic,Det lim__C20:4ω6 Arachidonic,"Det lim__C20:5ω3 (Eicosapentaenoic, EPA)",Det lim__C22:1,Det lim__C22:6ω3 (DHA),Lab__C14:0 Myristic,Lab__C15:0 Pentadecylic,Lab__C16:0 Palmitic,Lab__C16:1ω7 Palmitoleic,Lab__C18:0 Stearic,Lab__C18:1ω9 Oleic,Lab__C18:2ω6 Linoleic,Lab__C18:3ω3 α-Linolenic,Lab__C18:3ω6 γ-Linolenic,Lab__C20:0 Arachidic,Lab__C20:4ω6 Arachidonic,"Lab__C20:5ω3 (Eicosapentaenoic, EPA)",Lab__C22:1,Lab__C22:6ω3 (DHA),Date__C14:0 Myristic,Date__C15:0 Pentadecylic,Date__C16:0 Palmitic,Date__C16:1ω7 Palmitoleic,Date__C18:0 Stearic,Date__C18:1ω9 Oleic,Date__C18:2ω6 Linoleic,Date__C18:3ω3 α-Linolenic,Date__C18:3ω6 γ-Linolenic,Date__C20:0 Arachidic,Date__C20:4ω6 Arachidonic,"Date__C20:5ω3 (Eicosapentaenoic, EPA)",Date__C22:1,Date__C22:6ω3 (DHA),Method__C14:0 Myristic,Method__C15:0 Pentadecylic,Method__C16:0 Palmitic,Method__C16:1ω7 Palmitoleic,Method__C18:0 Stearic,Method__C18:1ω9 Oleic,Method__C18:2ω6 Linoleic,Method__C18:3ω3 α-Linolenic,Method__C18:3ω6 γ-Linolenic,Method__C20:0 Arachidic,Method__C20:4ω6 Arachidonic,"Method__C20:5ω3 (Eicosapentaenoic, EPA)",Method__C22:1,Method__C22:6ω3 (DHA),Remarks__C14:0 Myristic,Remarks__C15:0 Pentadecylic,Remarks__C16:0 Palmitic,Remarks__C16:1ω7 Palmitoleic,Remarks__C18:0 Stearic,Remarks__C18:1ω9 Oleic,Remarks__C18:2ω6 Linoleic,Remarks__C18:3ω3 α-Linolenic,Remarks__C18:3ω6 γ-Linolenic,Remarks__C20:0 Arachidic,Remarks__C20:4ω6 Arachidonic,"Remarks__C20:5ω3 (Eicosapentaenoic, EPA)",Remarks__C22:1,Remarks__C22:6ω3 (DHA)
0,#3847,Acinetospora crinita,Freeze dried,ECN Phyllis classification > Marine biomass (a...,TNO (Netherlands),2023-02-15 08:37:39,Phylum: Ochrophyta ^ Genus: Acinetospora ^ Spe...,Year: 2016 ^ Journal/project: Phycological Soc...,14.82,1.31,34.15,16.19,2.76,7.14,4.36,5.06,0.58,0.57,2.26,3.49,0.95,0.04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

In [143]:
# write the dataframe to excel
# combined_df.to_excel('#3847.xlsx', index=False)

In [154]:
# perform the same operations for all the files in the file_paths
error_files = []
combined_df = pd.DataFrame()
for i, file_path in enumerate(file_paths):
    try:
        df = pd.read_excel(file_path, engine='openpyxl')
    except:
        error_files.append(file_path)
    updf = df.iloc[:df.index[df.iloc[:, 0] == "Values"][0], :]
    downdf = df.iloc[df.index[df.iloc[:, 0] == "Values"][0]:, :]
    updf_T = updf.T.iloc[0:2]
    updf_T = updf_T.dropna(axis=1, how='all')
    updf_T.columns = range(updf_T.shape[1])
    updf_T = updf_T.drop([0, 1], axis=1)
    updf_T.columns = range(updf_T.shape[1])
    updf_T = updf_T.fillna(method='ffill', axis=1)
    updf_T_dict = dict()
    for i, col in enumerate(updf_T.iloc[0]):
        if col not in updf_T_dict:
            updf_T_dict[col] = updf_T.iloc[1, i]
        else:
            updf_T_dict[col] = updf_T_dict[col] + ' ^ ' + updf_T.iloc[1, i]
    updf_T_new = pd.DataFrame(updf_T_dict, index=[i])
    downdf_T = downdf.T
    downdf_T.columns = range(downdf_T.shape[1])
    downdf_T[1] = downdf_T[1].fillna(method='ffill', axis=0)
    downdf_T[1] = downdf_T[1].fillna('') + '_' + downdf_T[2].fillna('')
    downdf_T = downdf_T.drop([2], axis=1)
    downdf_T.columns = range(downdf_T.shape[1])
    downdf_ = downdf_T.copy()
    downdf_.columns = downdf_.iloc[0]
    downdf_ = downdf_.drop(downdf_.index[0:2])
    downdf_ = downdf_.dropna(axis=1, how='all')
    prefix_col = downdf_['Property_'].values.tolist()
    parameter_col = downdf_.columns[1:]
    dict_ = dict()
    for row in prefix_col:
        for col in parameter_col:
            dict_[row + '_' + col] = downdf_[downdf_['Property_'] == row][col].values[0]
    try:
        downdf_new = pd.DataFrame(dict_, index=[i])
    except:
        downdf_new = pd.DataFrame(dict_, index=[i, i+1])
    combined_data = pd.concat([updf_T_new, downdf_new], axis=1)
    # append the combined_data to the combined_df
    combined_df = combined_df.append(combined_data)
    

    

print(error_files)

['Data_database\\Data_database1\\types\\~$algae-chlorella_#1384.xlsx', 'Data_database\\Data_database1\\types\\~$algae-Synechococcus_#2327.xlsx', 'Data_database\\Data_database1\\types\\grass\\plant\\hemp\\~$hemp-(Cannabis-sativa-L.)-whole-plant_#1702.xlsx']


In [156]:
combined_df.shape

(3340, 1821)

In [158]:
# column_names are from updf_T_new
column_names = updf_T_new.columns
column_names

Index(['ID-number', 'Material', 'Classification', 'Ash type',
       'Submitter organisation', 'Remarks', 'Literature'],
      dtype='object')

In [159]:
# forwafill based on the column names
combined_df[column_names] = combined_df[column_names].fillna(method='ffill', axis=1)


In [161]:
combined_df.to_excel('combined.xlsx', index=False)